In [1]:
# STEP 3: RECOMMENDATION ENGINE

import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans


class RestaurantRecommender:
    """Restaurant Recommendation System"""

    def __init__(self, cleaned_data_path, encoded_data_path, encoder_path):
        print("🚀 Initializing Recommendation System...")

        # Load datasets
        self.df_cleaned = pd.read_csv(cleaned_data_path)
        self.df_encoded = pd.read_csv(encoded_data_path)

        # Load preprocessing objects
        with open(encoder_path, "rb") as f:
            obj = pickle.load(f)
            self.encoder = obj["encoder"]
            self.scaler = obj["scaler"]

        # Ensure index alignment
        self.df_encoded.index = self.df_cleaned.index

        # Similarity matrix (⚠ heavy but OK for demo)
        print("🧮 Computing cosine similarity...")
        self.similarity_matrix = cosine_similarity(self.df_encoded)

        # Clustering
        self.perform_clustering()

        print(f"✅ System initialized with {len(self.df_cleaned)} restaurants")

    def perform_clustering(self, n_clusters=10):
        """K-Means clustering"""
        print(f"\n🎯 Performing K-Means Clustering ({n_clusters} clusters)")

        kmeans = KMeans(
            n_clusters=n_clusters,
            random_state=42,
            n_init=10
        )

        self.df_cleaned["cluster"] = kmeans.fit_predict(self.df_encoded)

        print("✅ Clustering completed")
        print(self.df_cleaned["cluster"].value_counts().sort_index())

    def get_recommendations_by_similarity(self, restaurant_index, n_recommendations=5):
        """Cosine similarity based recommendations"""

        if restaurant_index >= len(self.df_cleaned):
            raise IndexError("Restaurant index out of range")

        scores = self.similarity_matrix[restaurant_index]
        indices = scores.argsort()[::-1][1:n_recommendations + 1]

        result = self.df_cleaned.iloc[indices].copy()
        result["similarity_score"] = scores[indices]

        return result

    def get_recommendations_by_filters(
        self,
        city=None,
        cuisine=None,
        min_rating=None,
        max_cost=None,
        n_recommendations=10
    ):
        """Filter-based recommendations"""

        df = self.df_cleaned.copy()

        if city:
            df = df[df["city"].str.lower() == city.lower()]

        if cuisine:
            df = df[df["cuisine"].str.lower().str.contains(cuisine.lower(), na=False)]

        if min_rating is not None:
            df = df[df["rating"] >= min_rating]

        if max_cost is not None:
            df = df[df["cost"] <= max_cost]

        return df.sort_values("rating", ascending=False).head(n_recommendations)

    def get_recommendations_by_cluster(self, restaurant_index, n_recommendations=5):
        """Same-cluster recommendations"""

        cluster_id = self.df_cleaned.iloc[restaurant_index]["cluster"]

        df = self.df_cleaned[self.df_cleaned["cluster"] == cluster_id]
        df = df[df.index != restaurant_index]

        return df.sort_values("rating", ascending=False).head(n_recommendations)

    def get_top_rated_restaurants(self, city=None, n_recommendations=10):
        """Top-rated restaurants"""

        df = self.df_cleaned.copy()

        if city:
            df = df[df["city"].str.lower() == city.lower()]

        return df.sort_values("rating", ascending=False).head(n_recommendations)

    def get_budget_friendly_restaurants(self, city=None, n_recommendations=10):
        """Budget-friendly restaurants"""

        df = self.df_cleaned.copy()

        if city:
            df = df[df["city"].str.lower() == city.lower()]

        return df.sort_values(["cost", "rating"], ascending=[True, False]).head(n_recommendations)

    def search_restaurant_by_name(self, name):
        """Search restaurant by name"""

        return self.df_cleaned[
            self.df_cleaned["name"].str.lower().str.contains(name.lower(), na=False)
        ]


def demo_recommendations():
    print("\n" + "🎯 RESTAURANT RECOMMENDATION DEMO 🎯".center(60))

    # ✅ UPDATED PATHS
    cleaned_file = r"D:\Py_start\Python\project_SN\Project4\1\swiggy_cleaned_data.csv"
    encoded_file = r"D:\Py_start\Python\project_SN\Project4\1\swiggy_encoded_data.csv"
    encoder_file = r"D:\Py_start\Python\project_SN\Project4\models\preprocessing.pkl"

    try:
        recommender = RestaurantRecommender(
            cleaned_file,
            encoded_file,
            encoder_file
        )

        # Demo 1: Similarity
        print("\n📌 SIMILARITY BASED RECOMMENDATIONS")
        base_idx = 0
        print(recommender.df_cleaned.iloc[base_idx][
            ["name", "city", "cuisine", "rating", "cost"]
        ])

        similar = recommender.get_recommendations_by_similarity(base_idx, 5)
        print(similar[["name", "city", "cuisine", "rating", "cost", "similarity_score"]])

        # Demo 2: Filter
        print("\n📌 FILTER BASED RECOMMENDATIONS")
        filtered = recommender.get_recommendations_by_filters(
            city="Bangalore",
            min_rating=4.0,
            max_cost=500,
            n_recommendations=5
        )
        print(filtered[["name", "cuisine", "rating", "cost"]])

        # Demo 3: Top rated
        print("\n📌 TOP RATED RESTAURANTS")
        print(recommender.get_top_rated_restaurants(5)[
            ["name", "city", "rating", "cost"]
        ])

        # Demo 4: Budget friendly
        print("\n📌 BUDGET FRIENDLY RESTAURANTS")
        print(recommender.get_budget_friendly_restaurants(5)[
            ["name", "city", "rating", "cost"]
        ])

        print("\n🎉 DEMO COMPLETED SUCCESSFULLY 🎉")

    except Exception as e:
        print("❌ ERROR:", e)


if __name__ == "__main__":
    demo_recommendations()



             🎯 RESTAURANT RECOMMENDATION DEMO 🎯             
🚀 Initializing Recommendation System...
🧮 Computing cosine similarity...
❌ ERROR: Unable to allocate 164. GiB for an array with shape (148442, 148442) and data type float64
